# Module 10 - Week 03
# Segmenting and Clustering Neighborhoods in Toronto

In [80]:
# import modules and libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

!pip install geocoder
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
! pip install folium==0.5.0
import folium # map rendering library

!pip install beautifulsoup4
from bs4 import BeautifulSoup

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [81]:
# Question 1 - Import  data from wikipedia

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


df.head(11)
# df['Borough'].value_counts()
# df.groupby(['PostalCode'])['Neighborhood'].value_counts(normalize=True)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [82]:

# Question 2 - read the csv file
geo = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
new_df = pd.merge(df, geo, how='left', left_on='PostalCode', right_on='Postal Code')
new_df



,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494
...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",M8X,43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,M4Y,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,M7Y,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",M8Y,43.636258,-79.498509


In [83]:
# Remove Unassigned postal codes if any
df1=df[df['Borough']!='Not assigned']

df2 = df1.groupby(['PostalCode','Borough'], sort=False).agg(', '.join)
df2.reset_index(inplace=True)

# Print the result
df2.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [85]:
# Consider boroughs containing - "Toronto"  

df_toronto = new_df[new_df['Borough'].str.contains('Toronto')]
df_toronto.head(9)


,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,M5G,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,M6G,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",M5H,43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",M6H,43.669005,-79.442259


In [86]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df_toronto['Latitude'],df_toronto['Longitude'],df_toronto['Borough'],df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

In [87]:
# set number of clusters
kclusters = 5

toronto_clustering = df_toronto.drop(['PostalCode','Borough','Neighborhood', 'Postal Code'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# add clustering labels
df_toronto.insert(0, 'Cluster Labels', kmeans.labels_)

df_toronto.tail() # check the last columns!

,Cluster Labels,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
92,4,M5W,Downtown Toronto Stn A,Enclave of M5E,M5W,43.646435,-79.374846
96,4,M4X,Downtown Toronto,"St. James Town, Cabbagetown",M4X,43.667967,-79.367675
97,4,M5X,Downtown Toronto,"First Canadian Place, Underground city",M5X,43.648429,-79.382280
99,4,M4Y,Downtown Toronto,Church and Wellesley,M4Y,43.665860,-79.383160
100,1,M7Y,East Toronto Business,Enclave of M4L,M7Y,43.662744,-79.321558


In [88]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood'], df_toronto['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [89]:
CLIENT_ID = 'CNSTOEOZL4JT2MYF3JQGPIQDPNZWZD5HLMLG1XFYJ5Y5DFHQ' # your Foursquare ID
CLIENT_SECRET = 'OT0MOCBKW4JHMWKLB1ZSCVMMNLWCXURO3ZLWVVKEJIAD1QYM' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: CNSTOEOZL4JT2MYF3JQGPIQDPNZWZD5HLMLG1XFYJ5Y5DFHQ
CLIENT_SECRET:OT0MOCBKW4JHMWKLB1ZSCVMMNLWCXURO3ZLWVVKEJIAD1QYM


In [90]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [91]:

toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [94]:
# toronto_venues.count()
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
Central Bay Street,62,62,62,62,62,62
Christie,16,16,16,16,16,16
Church and Wellesley,81,81,81,81,81,81
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35
Davisville North,10,10,10,10,10,10


In [95]:
# Explore the venues in the neighborhoods

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(9)

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.017241,0.0,0.0,0.00,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000
3,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.016129,0.0,0.0,0.00,0.0,0.016129
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000
5,Church and Wellesley,0.012346,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.024691
6,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.0,...,0.0,0.000000,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.000000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.0,...,0.0,0.028571,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [96]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(9)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Bakery,Coffee Shop,Cheese Shop,Restaurant,Pharmacy,Pub,Seafood Restaurant,Farmers Market,Beer Bar
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Convenience Store,Intersection,Bar,Bakery,Restaurant,Climbing Gym,Furniture / Home Store
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Plane,Boutique,Boat or Ferry,Rental Car Location,Bar,Coffee Shop
3,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Restaurant,Burger Joint,Bubble Tea Shop,Japanese Restaurant,Salad Place,Portuguese Restaurant
4,Christie,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Candy Store,Coffee Shop
5,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Pub,Men's Store,Mediterranean Restaurant,Smoke Shop,Hotel
6,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Gym,Italian Restaurant,Bakery,Japanese Restaurant,Seafood Restaurant,Deli / Bodega
7,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Café,Italian Restaurant,Thai Restaurant,Sushi Restaurant,Gym,Coffee Shop,Pharmacy
8,Davisville North,Gym / Fitness Center,Hotel,Gym,Park,Playground,Convenience Store,Breakfast Spot,Department Store,Sandwich Place,Food & Drink Shop


In [97]:
toronto_merged = df_toronto
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
# toronto_merged.value_counts()
# toronto_merged['1st Most Common Venue'].value_counts(normalize=True)
# toronto_merged.loc[toronto_merged['1st Most Common Venue'] == 'Airport Service']
# toronto_merged.groupby(['1st Most Common Venue'])['Café'].value_counts(normalize=True)

toronto_merged.head(9)

,Cluster Labels,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Cosmetics Shop,Shoe Store,Restaurant
9,4,M5B,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937,Coffee Shop,Clothing Store,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Café,Cosmetics Shop,Lingerie Store,Bookstore,Ramen Restaurant
15,4,M5C,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418,Café,Coffee Shop,Cosmetics Shop,Restaurant,Cocktail Bar,Bakery,Seafood Restaurant,Clothing Store,Moroccan Restaurant,Park
19,1,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031,Health Food Store,Pub,Trail,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Electronics Store
20,4,M5E,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306,Cocktail Bar,Bakery,Coffee Shop,Cheese Shop,Restaurant,Pharmacy,Pub,Seafood Restaurant,Farmers Market,Beer Bar
24,4,M5G,Downtown Toronto,Central Bay Street,M5G,43.657952,-79.387383,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Restaurant,Burger Joint,Bubble Tea Shop,Japanese Restaurant,Salad Place,Portuguese Restaurant
25,0,M6G,Downtown Toronto,Christie,M6G,43.669542,-79.422564,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Candy Store,Coffee Shop
30,4,M5H,Downtown Toronto,"Richmond, Adelaide, King",M5H,43.650571,-79.384568,Coffee Shop,Café,Restaurant,Thai Restaurant,Clothing Store,Gym,Deli / Bodega,Pizza Place,Burrito Place,Sushi Restaurant
31,3,M6H,West Toronto,"Dufferin, Dovercourt Village",M6H,43.669005,-79.442259,Bakery,Pharmacy,Café,Brewery,Pet Store,Wine Shop,Grocery Store,Park,Bank,Bar


In [98]:
# Cluster the neighborhoods
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# Execute k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [99]:
# Display geograpical coordinates of Toronto 

address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [100]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters